In [25]:
#packages
import pandas as pd
# !pip install folium
from folium.plugins import Geocoder
import folium
from folium import plugins
import s3fs

In [26]:
# Define your S3 bucket and folder

# Define the full S3 path to the file
s3_file_path = 's3://keepaustinweird/austin_key/data.csv'

# Read the file into a Pandas DataFrame
filtered_df = pd.read_csv(s3_file_path)

# Display the DataFrame
filtered_df.head()


,published_date,issue_reported,latitude,longitude,address,traffic_report_status,traffic_report_status_date_time
0,2024-01-24 23:32:05+00:00,Stalled Vehicle,0.000000,0.000000,200 Dedman,ARCHIVED,2024-01-25T00:15:11.000Z
1,2024-01-01 04:39:24+00:00,VEHICLE FIRE,30.376018,-97.501944,14614 BOIS D ARC RD,ARCHIVED,2024-01-01T05:10:08.000Z
2,2024-01-01 07:05:26+00:00,Crash Service,30.302704,-97.661269,Loyola Ln / Ed Bluestein Blvd Svrd Sb,ARCHIVED,2024-01-01T10:30:07.000Z
3,2024-01-01 10:53:42+00:00,Crash Service,30.385992,-97.672917,11900 N IH 35 NB,ARCHIVED,2024-01-01T18:00:09.000Z
4,2024-01-01 05:01:41+00:00,COLLISION,30.344979,-97.580056,10212-10256 E US 290 HWY SVRD WB,ARCHIVED,2024-01-01T06:15:08.000Z


In [27]:
len(filtered_df)

9155

In [28]:
filtered_df.sort_values(by="published_date", ascending=False)

,published_date,issue_reported,latitude,longitude,address,traffic_report_status,traffic_report_status_date_time
9085,2024-03-13 16:45:42+00:00,Traffic Hazard,30.197026,-97.760751,1800 E Stassney Ln,ACTIVE,2024-03-13T16:55:15.000Z
9154,2024-03-13 16:45:09+00:00,Stalled Vehicle,30.310519,-97.709332,5200-blk N Ih 35 Svrd Nb,ACTIVE,2024-03-13T16:50:19.000Z
3495,2024-03-13 16:45:09+00:00,Stalled Vehicle,30.310519,-97.709332,5200-blk N Ih 35 Svrd Nb,ACTIVE,2024-03-13T16:50:19.000Z
9153,2024-03-13 16:37:31+00:00,COLLISION,30.282229,-97.623206,0 GOODNESS WAY,ACTIVE,2024-03-13T16:40:16.000Z
3486,2024-03-13 16:37:31+00:00,COLLISION,30.282229,-97.623206,0 GOODNESS WAY,ACTIVE,2024-03-13T16:40:16.000Z
...,...,...,...,...,...,...,...
26,2024-01-01 00:45:18+00:00,Traffic Hazard,30.367062,-97.613624,12208-12236 Harris Branch Pkwy,ARCHIVED,2024-01-01T05:00:08.000Z
5055,2024-01-01 00:39:11+00:00,Stalled Vehicle,30.361197,-97.698042,9209 N LAMAR BLVD,ARCHIVED,2024-01-01T03:00:08.000Z
10,2024-01-01 00:39:11+00:00,Stalled Vehicle,30.361197,-97.698042,9209 N LAMAR BLVD,ARCHIVED,2024-01-01T03:00:08.000Z
5132,2024-01-01 00:01:25+00:00,Crash Urgent,30.189123,-97.767910,E William Cannon Dr / Bluff Springs Rd,ARCHIVED,2024-01-01T03:00:08.000Z


In [29]:
#MAPPING AUSTIN

#drop longitude and latitude where it is 0,0 and drop na values
value_to_drop = 0
filtered_df2 = filtered_df[(filtered_df['longitude'] != value_to_drop) & (filtered_df["latitude"] != value_to_drop)]
filtered_df2 = filtered_df2.dropna()

#creating a new df with just longitude, latitude, and incident type
mapping_df = filtered_df2[["issue_reported", "latitude", "longitude"]]
mapping_df.head()

,issue_reported,latitude,longitude
1,VEHICLE FIRE,30.376018,-97.501944
2,Crash Service,30.302704,-97.661269
3,Crash Service,30.385992,-97.672917
4,COLLISION,30.344979,-97.580056
5,Crash Urgent,30.356958,-97.679610


In [34]:
len(mapping_df[mapping_df['issue_reported']=='LOOSE LIVESTOCK'])

235

In [30]:
len(mapping_df)

9121

In [31]:
#dictinary of icons and colors
# Icon dictionary with Font Awesome icon names
icon_dict = {
    'Traffic Hazard': 'road-barrier',
    'Crash Urgent': 'car-burst',
    'TRFC HAZD/ DEBRIS': 'road-barrier',
    'Crash Service': 'truck',
    'COLLISION WITH INJURY': 'user-injured',
    'Stalled Vehicle': 'car',
    'COLLISION': 'car-burst',
    'LOOSE LIVESTOCK': 'cow',
    'COLLISN/ LVNG SCN': 'car-burst',
    'VEHICLE FIRE': 'fire',
    'COLLISION/PRIVATE PROPERTY': 'lock',
    'BLOCKED DRIV/ HWY': 'ban',
    'OBSTRUCT HWY': 'road-barrier',
    'AUTO/ PED': 'person-walking',
    'BOAT ACCIDENT': 'ship',
    'TRAFFIC FATALITY': 'face-dizzy',
    'FLEET ACC/ INJURY': 'user-injured',
    'ICY ROADWAY': 'icicles',
    'zSTALLED VEHICLE': 'car-side',
    'N / HZRD TRFC VIOL': 'circle-stop',
    'HIGH WATER': 'water',
    'Traffic Impediment': 'ban',
    'COLLISN / FTSRA': 'car-burst',
    'FLEET ACC/ FATAL': 'truck-medical'
}

color_dict = {
    'Traffic Hazard': 'orange',
    'Crash Urgent': 'red',
    'TRFC HAZD/ DEBRIS': 'orange',
    'Crash Service': 'darkblue',
    'COLLISION WITH INJURY': 'lightred',
    'Stalled Vehicle': 'lightgray',
    'COLLISION': 'red',
    'LOOSE LIVESTOCK': 'beige',
    'COLLISN/ LVNG SCN': 'red',
    'VEHICLE FIRE': 'darkred',
    'COLLISION/PRIVATE PROPERTY': 'purple',
    'BLOCKED DRIV/ HWY': 'darkblue',
    'OBSTRUCT HWY': 'orange',
    'AUTO/ PED': 'cadetblue',
    'BOAT ACCIDENT': 'blue',
    'TRAFFIC FATALITY': 'darkpurple',
    'FLEET ACC/ INJURY': 'lightgreen',
    'ICY ROADWAY': 'lightblue',
    'zSTALLED VEHICLE': 'lightgray',
    'N / HZRD TRFC VIOL': 'darkgreen',
    'HIGH WATER': 'lightblue',
    'Traffic Impediment': 'darkblue',
    'COLLISN / FTSRA': 'red',
    'FLEET ACC/ FATAL': 'purple'
}

In [32]:
# CREATING THE MAP
#map baseline
interactive_map = folium.Map(
    location=(30.266666, -97.733330),
    zoom_start=12,
    tiles="https://mt1.google.com/vt/lyrs=r&x={x}&y={y}&z={z}",
    attr="Google maps",
)

# Initialize a dictionary to hold feature groups for each incident type
incident_feature_groups = {}

# Create a Feature Group for each incident type and add markers to it
for index, row in mapping_df.iterrows():
    issue_reported = row['issue_reported']
    icon_name = icon_dict.get(issue_reported, 'question')
    marker_color = color_dict.get(issue_reported, 'gray')

    # Check if the feature group for this incident type already exists, create if not
    if issue_reported not in incident_feature_groups:
        incident_feature_groups[issue_reported] = folium.FeatureGroup(name=issue_reported, show=False)

    # Create the marker
    marker = folium.Marker(
        [row['latitude'], row['longitude']],
        popup=issue_reported,
        icon=folium.Icon(color=marker_color, icon=icon_name, prefix='fa')
    )

    # Add the marker to the respective feature group
    marker.add_to(incident_feature_groups[issue_reported])

# Add Heat Map Feature Group (as before)
heat_map_feature_group = folium.FeatureGroup(name='Heat Map', show=False)
heat_data = list(zip(mapping_df.latitude, mapping_df.longitude))
plugins.HeatMap(heat_data).add_to(heat_map_feature_group)
heat_map_feature_group.add_to(interactive_map)

# Add each incident feature group to the map
for fg in incident_feature_groups.values():
    fg.add_to(interactive_map)

# Optionally, add a MiniMap (as before)
minimap = plugins.MiniMap(position='bottomleft')
interactive_map.add_child(minimap)

# Add Layer Control to manage visibility of each layer
folium.LayerControl().add_to(interactive_map)

Geocoder().add_to(interactive_map)

# Display the map
# interactive_map

In [33]:
#get html of interactive map
interactive_map.save("austin_incidents.html")